In [27]:
import numpy as np
import matplotlib.pyplot as plt
# FDM solution to 1D Euler Equations
# Initial Conditions - 1D sod shock tube initial conditions
# Dirichlet Boundary Conditions
# Constants
import numpy as np
import matplotlib.pyplot as plt

# Constants
gamma = 1.4

# initial conditions: (rho, u, p)_{t = 0} = (1, 0, 1) if x in (0, 0.5] and (0.125, 0, 0.1) if x in (0.5, 1.0)
def sod_initial_conditions(nx, x0=0.5, x_min=0.0, x_max=1.0):
    x = np.linspace(x_min, x_max, nx)
    rho = np.where(x < x0, 1.0, 0.125)
    u = np.zeros(nx)
    p = np.where(x < x0, 1.0, 0.1)
    return x, rho, u, p

# pressure related to conserved quantities through the equation of state given by p = (gamma - 1)(E - 1/2 pu^2)
# gamma = 1.4
# this method and next method are convert between energy and pressure
def primitive_to_conservative(rho, u, p):
    E = p / (gamma - 1) + 0.5 * rho * u**2
    return np.array([rho, rho * u, E])

def conservative_to_primitive(U):
    rho = U[0]
    u = U[1] / rho
    E = U[2]
    p = (gamma - 1) * (E - 0.5 * rho * u**2)
    return rho, u, p


def compute_flux(U):
    rho, u, p = conservative_to_primitive(U)
    E = U[2]
    return np.array([
        rho * u,
        rho * u**2 + p,
        u * (E + p)
    ])

def rusanov_flux(U_L, U_R):
    flux_L = compute_flux(U_L)
    flux_R = compute_flux(U_R)

    rho_L, u_L, p_L = conservative_to_primitive(U_L)
    rho_R, u_R, p_R = conservative_to_primitive(U_R)

    c_L = np.sqrt(gamma * p_L / rho_L)
    c_R = np.sqrt(gamma * p_R / rho_R)

    s_max = max(abs(u_L) + c_L, abs(u_R) + c_R)

    return 0.5 * (flux_L + flux_R) - 0.5 * s_max * (U_R - U_L)

def apply_boundary_conditions(U):
    # Simple transmissive (zero gradient) boundaries
    U[:, 0] = U[:, 1]
    U[:, -1] = U[:, -2]
    return U

In [ ]:
def sod_solver(nx=400, CFL=0.75, t_end=0.1):
    x, rho, u, p = sod_initial_conditions(nx)
    dx = x[1] - x[0]

    U = primitive_to_conservative(rho, u, p)
    t = 0.0

    while t < t_end:
        U = apply_boundary_conditions(U)
        rho, u, p = conservative_to_primitive(U)
        c = np.sqrt(gamma * p / rho)
        dt = CFL * dx / np.max(np.abs(u) + c)
        if t + dt > t_end:
            dt = t_end - t

        # Compute fluxes at interfaces
        F = np.zeros_like(U)
        for i in range(1, nx - 1):
            F[:, i] = rusanov_flux(U[:, i], U[:, i + 1])
            #F[:, i] = hllc_flux(U[:, i], U[:, i + 1])

        # Update U
        U[:, 1:-1] -= dt / dx * (F[:, 1:-1] - F[:, 0:-2])

        t += dt

    rho, u, p = conservative_to_primitive(U)
    return x, rho, u, p

def plot_solution(x, rho, u, p):
    plt.figure(figsize=(12, 8))

    plt.subplot(3, 1, 1)
    plt.plot(x, rho)
    plt.ylabel('Density')

    plt.subplot(3, 1, 2)
    plt.plot(x, u)
    plt.ylabel('Velocity')

    plt.subplot(3, 1, 3)
    plt.plot(x, p)
    plt.ylabel('Pressure')
    plt.xlabel('x')

    plt.tight_layout()
    plt.show()

# Run and Plot 
x, rho, u, p = sod_solver()
plot_solution(x, rho, u, p)
## TO DO: Comparison FDM solution to Analytic Solution 